# chapter 

## Setting
- GPT 모델 설정 필요

In [1]:
import os
key_file = open('key.txt', 'r')
os.environ['OPENAI_API_KEY'] = key_file.readline()
key_file.close()
from openai import OpenAI
client = OpenAI()
model = "o4-mini"

In [2]:
%%time

messages = []

system_content = '''
# 게임 설명
메이즈 게임은 미로 찾기 게임이고, 파이썬으로 아바타가 도착점으로 가도록 만들어야 하는 게임이다
'''

user_content = '''
'''

messages.append({"role": "system", "content": system_content})
messages.append({"role": "user", "content": user_content})

response = client.chat.completions.create(
    model=model,
    messages=messages)
messages.append(response.choices[0].message)

print('*'*20, 'answer', '*'*20)
print(response.choices[0].message.content)
print('*'*40)
print('tokens:', response.usage)

******************** answer ********************
안녕하세요! 메이즈 게임 구현을 도와드리겠습니다. 우선 몇 가지 사항을 정하고 시작하면 좋을 것 같습니다.

1. 실행 환경
  - 텍스트(콘솔) 기반인가요, 아니면 Pygame 같은 그래픽 라이브러리를 사용할 계획인가요?  
2. 미로 생성 방식
  - 미로를 랜덤 생성할 것인지, 아니면 미리 정의된 맵을 불러올 것인지요?  
3. 아바타 조작
  - 키보드 방향키로 직접 움직이게 할 건가요, 아니면 자동 경로 탐색(예: BFS/DFS)로 도착점까지 가도록 할 건가요?  
4. 추가 기능
  - 타이머, 아이템, 점수 시스템 등 부가 요소가 필요한지요?

위 정보를 알려주시면 예제 코드 구조나 구현 방안을 구체적으로 제안해 드리겠습니다. 어떤 형태로 진행할지 알려주세요!
****************************************
tokens: CompletionUsage(completion_tokens=359, prompt_tokens=49, total_tokens=408, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=128, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
CPU times: total: 31.2 ms
Wall time: 5.96 s


In [3]:
%%time

user_content = ''' 
# 명령어 설명 
- moveForward(): 앞으로 1칸 전진.
- turnLeft(): 좌측으로 90도 회전.
- turnRight(): 우측으로 90도 회전.
- turnBack(): 180도 회전.
- notDone(): 아바타가 도착지에 도착한 경우
- pathAhead(): 앞으로 이동 가능한 경우
- pathRight(): 우측으로 이동 가능한 경우
- pathLeft(): 좌측으로 이동 가능한 경우
이해했으면 이해했습니다라고 말한다
'''

messages.append({"role": "user", "content": user_content})

response = client.chat.completions.create(
    model=model,
    messages=messages)
messages.append(response.choices[0].message)

print('*'*20, 'answer', '*'*20)
print(response.choices[0].message.content)
print('*'*40)
print('tokens:', response.usage)

******************** answer ********************
이해했습니다
****************************************
tokens: CompletionUsage(completion_tokens=86, prompt_tokens=384, total_tokens=470, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=64, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
CPU times: total: 31.2 ms
Wall time: 1.63 s


In [4]:
%%time

user_content = ''' 
<질문 프롬프트>

[0,0,g,1,1,0,0,0]
[0,0,0,0,1,0,0,0]
[0,0,0,0,1,1,0,0]
[0,0,0,0,0,1,1,1]
[s,1,0,0,0,0,1,0]
[0,1,1,0,0,0,1,0]
[0,0,1,0,0,0,1,0]
[0,1,1,1,1,1,1,0]
[0,0,1,0,0,0,0,0]

s가 아바타 시작위치이고  g가 goal이야
아바타는 시작 위치에서 오른쪽을 바라보고 있어
0은 길이 없는 곳이고 1은 길이 있는 곳이야
아바타가 g까지 갈 수 있도록 파이썬 코드를 작성해줘

'''

messages.append({"role": "user", "content": user_content})

response = client.chat.completions.create(
    model=model,
    messages=messages)
messages.append(response.choices[0].message)

print('*'*20, 'answer', '*'*20)
print(response.choices[0].message.content)
print('*'*40)
print('tokens:', response.usage)

******************** answer ********************
아래 코드는 오른손 우선(right‐hand rule) 알고리즘을 이용해 아바타가 골 지점(g)까지 이동하게 합니다.  

```python
def solveMaze():
    # 골에 도착할 때까지 반복
    while notDone():
        # 1) 오른쪽 뱡향으로 갈 수 있으면
        if pathRight():
            turnRight()
            moveForward()
        # 2) 아니면 앞이 뚫렸으면
        elif pathAhead():
            moveForward()
        # 3) 아니면 왼쪽으로
        elif pathLeft():
            turnLeft()
            moveForward()
        # 4) 그 외(막다른 길)이면 뒤돌아서
        else:
            turnBack()
            moveForward()

# 실행
solveMaze()
```

설명:  
1. pathRight(), pathAhead(), pathLeft() 함수로 주변을 탐색  
2. 가능한 방향에 맞춰 회전(turnRight/turnLeft/turnBack) 후 moveForward()  
3. 골에 도달하면 notDone()가 False가 되어 루프 종료  
4. 시작 방향은 오른쪽(동쪽)으로 가정됨  

이 코드만 넣으면 주어진 미로에서 s→g를 자동으로 찾아갑니다.
****************************************
tokens: CompletionUsage(completion_tokens=803, prompt_tokens=623, total_tokens=1426, completion_tokens_details=CompletionTokensDetails(accepted_predictio